## Azure API Management from a Developer Perspective

### Preparation

* Log into the right Azure subscription

In [ ]:
az login --output none
az account set --subscription 99da81c1-72ed-4a56-8c96-f360dbd22610 --output none

* Create a copy of the Real Estate **Infra**

In [ ]:
Write-Host "Navigate to the infra"
cd ../infra/04_applications/real-estate-app
Write-Host "Create resource group"
az group create --name ftw-dem-real-estate-app-rg --location westeurope --output none
Write-Host "Deploy the infra"
./deploy.ps1 -EnvironmentName "dem" -EnvironmentKey "dev" -ApplyIpRetrictions "false"
Write-Host "Navigate back"
cd ../../../demos

* Build and deploy the Real Estate **Sales App**

In [ ]:
Write-Host "Navigate to source"
cd ../src/real-estate-app/Ftw.RealEstate.Sales
Write-Host "Build Sales App"
dotnet restore
dotnet build --configuration Release --no-restore
dotnet publish --configuration Release --no-restore --output ./output
Compress-Archive ./output/* ./output/app.zip -Update
Write-Host "Deploy Sales App"
az webapp deployment source config-zip --resource-group ftw-dem-real-estate-app-rg --name ftw-dem-real-estate-sales-wapp --src ./output/app.zip
Write-Host "Navigate back"
cd ../../../demos

* Build and deploy the Real Estate **Rental App**

In [ ]:
Write-Host "Navigate to source"
cd ../src/real-estate-app/Ftw.RealEstate.Rental
Write-Host "Build Rental App"
dotnet restore
dotnet build --configuration Release --no-restore
dotnet publish --configuration Release --no-restore --output ./output
Compress-Archive ./output/* ./output/app.zip -Update
Write-Host "Deploy Rental App"
az webapp deployment source config-zip --resource-group ftw-dem-real-estate-app-rg --name ftw-dem-real-estate-rental-wapp --src ./output/app.zip
Write-Host "Navigate back"
cd ../../../demos

* Open the correct browser tabs

In [ ]:
Write-Host "Open the right browser tabs: started..."
start microsoft-edge:https://portal.azure.com/#@toonvanhouttegmail.onmicrosoft.com/resource/subscriptions/99da81c1-72ed-4a56-8c96-f360dbd22610/resourceGroups/ftw-dem-real-estate-app-rg/overview
start microsoft-edge:https://ftw-dem-real-estate-sales-wapp.azurewebsites.net/swagger/index.html
start microsoft-edge:https://ftw-dem-real-estate-rental-wapp.azurewebsites.net/graphql
start microsoft-edge:https://portal.azure.com/#@toonvanhouttegmail.onmicrosoft.com/resource/subscriptions/99da81c1-72ed-4a56-8c96-f360dbd22610/resourceGroups/ftw-dev-shared-infrastructure-rg/providers/Microsoft.ApiManagement/service/ftw-dev-api-mgmt-apim/overview
Write-Host "Open the right browser tabs: started..."

### It allows you to change implementation

* Create a new Real Estate Demo API, named real-estate-demo-api

* Create API from the Sales API Swagger spec: https://ftw-dem-real-estate-sales-wapp.azurewebsites.net/swagger/v1/swagger.json

* Rename the API operations, by adding *for sale*

* Show API settings

* Disable the subscription key

* Test the API in Postman

### It allows you to mash-up new APIs

* Create a new operation **Get rental apartments** with GET /rental/apartments

* Test the API and receive a 404

* Configure the following **inbound** policy

```xml
<!--Set URL of Rental API -->
<set-backend-service base-url="https://ftw-dem-real-estate-rental-wapp.azurewebsites.net/graphql" />
```

* Test the API and receive a 400

* Extend the **inbound** policy

```xml
<!--Transform HTTP Request into GraphQL Query-->
<set-method>POST</set-method>
<rewrite-uri template="/" copy-unmatched-params="false" />
<set-header name="Content-Type" exists-action="override">
    <value>application/json</value>
</set-header>
<set-body template="none">{
    "query": "query getAllApartments{ apartments { city description price hasBalcony } }"
}</set-body>
```

* Test the API and receive data response

* Configure the following **outbound** policy

```xml
<!--Transform HTTP Response to only return the apartments array-->
<set-body template="none">@(context.Response.Body.As<JObject>().SelectToken("data.apartments").ToString())</set-body>
```

* Test the API and receive desired response

### It allows you to mock API endpoints

* Show the mock file on the storage account

* Create a new operation **Get rental houses** with GET /rental/houses

* Configure the following **inbound** policy

```xml
<!--Transform request to query Blob Storage-->
<set-method>GET</set-method>
<set-backend-service base-url="https://ftwdemrealestatest.blob.core.windows.net/mocking" />
<rewrite-uri template="/rental-houses.json" copy-unmatched-params="false" />
```

* Test API and receive 404

* Extend the **inbound** policy

```xml
<!--Configure authentication required for Blob Storage-->
<set-header name="x-ms-version" exists-action="override">
    <value>2017-11-09</value>
</set-header>
<authentication-managed-identity resource="https://storage.azure.com" />
```

* Test the API and receive a non-JSON response

* Configure the following **outbound** policy

```xml
<!--Transform HTTP Response to application/json-->
<set-header name="Content-Type" exists-action="override">
    <value>application/json</value>
</set-header>
```

* Test the API and receive a JSON response

### It allows you to have end-to-end traceability

* Configure an App Insights Logger that points to the Real Estate instance

* Enable it on the Demo API

* Do a call and copy already a Request Id

* Show *Live Metrics*

* Search transaction with *Request Id*

### It allows you to harden your security

* Add IP restriction on the Web Apps

* Try system API and receive *Forbidden*

* Show the real API and the JWT Validation

* Try Demo API and receive *Unauthorized*

### It allows you to get broader adoption

* Log into Developer Portal in private mode (demo@yourazurecoach.com)

* Navigate to the real estate API

* Show JWT Token

* Try GET API, receive *OK*

* Try POST API, receive *Unauthorized*

### One more thing: security + monitoring

* Call the Hotels API via Postman

* Get Ftw-Request-Id and look into the logging

* Rename the capacity field and enable the *Remove stacktraces* via the named values

* See the exception message you get

TO Improve:
* Real end-to-end tracing with SQL Database?
* Create API Management Dashboard based on Log Analytics
* Fix the Developer Portal navigation
* Automatically blob reader/writer access on storage account